## Task: Create interesting and interactive data visualizations using the NOAA Climate Database. 

## 1. Create a Database
Using the instructions from the homework, I first created a database with three tables: temperatures, stations, and countries. 

To create a database from the NOAA climate data, I used sqlite3 to create the database and to query the data, as well as pandas to manipulate the dataframes. 
 
 ```python 
# importing the libraries  
import sqlite3 
import pandas as pd
import numpy as np 

# Importing the csv file 
temps_df = pd.read_csv("temps.csv")
temps_df.head()

 ```
 
Next, the temperature database had to be cleaned. The prepare_df reorganizes the tempperature data so that it can be easily used. 

```python
def prepare_df(df):
    df = df.set_index(keys=["ID", "Year"])
    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)
```

Now, we are going to start creating the connection into the climate database. As per the homework, our database will be called *climate-database.db*. 

```python
conn = sqlite3.connect("climate-database.db") # temperature database
```

:::{.callout-note} 
We realize that the temerature data set contains many rows, so it is better to have the data be loaded into the database in chunks. The following loop iterates through the 100000 rows of the data at a time, cleaning it with `prepare_df` and adding it to the database. 
:::

```python
temps_iter = pd.read_csv("temps.csv", chunksize = 100000)

for i, temps_df in enumerate(temps_iter): 
    df = prepare_df(temps_df)
    df.to_sql("temperatures", conn, if_exists="replace" if i == 0 else "append", index = False)
```

Since the temerature table is ready, we will start to read in the data from the station and country and add them as individual tables into the database. 

```python
# Adding the stations table
stations = pd.read_csv("station-metadata.csv") 
stations.to_sql("stations", conn, if_exists = "replace", index = False)

# Adding the countries table
countries = pd.read_csv("countries.csv")
countries.to_sql("countries", conn, if_exists = "replace", index = False)
```

To make sure that the tables are created, we will use a cursor to look into the SQL table. 

```python
cursor = conn.cursor() 

cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])
```

You should get something like this: 

![Output](Output.png){width=250}


Once you have this, you have successfully created a climate database. 

## 2. Writing a Query 
In order for us to access the `climate-database`, we need to write a SQL query. I used the following query to access key information that the homework is asking us to retrieve: 

```python
import sqlite3
import pandas as pd 

def query_climate_database(db_file, country, year_begin, year_end, month): 
    
    conn = sqlite3.connect(db_file)

    query = f'''
                SELECT S.name, S.latitude, S.longitude, C.name, T.year, T.month, T.temp
                FROM temperatures T 
                LEFT JOIN stations S on T.id = S.id
                LEFT JOIN countries C on SUBSTRING(T.id, 1, 2) = C.'FIPS 10-4'
                WHERE T.year >= {year_begin} AND T.year <= {year_end} AND T.month == {month} AND C.name == "{country}"
                '''

    df = pd.read_sql_query(query, conn)

    conn.close()
    return df 

```

To keep it short, here is a quick run down about what this query is doing: \
`SELECT` - Selecting the name, latitude, and longitude from STATIONS; Name from COUNTRIES; Year, Month, and Temp from TEMPERATURES \
`FROM` - Temperatures table (Aliased as T for readability purposes) \ \
`LEFT JOIN` - IDs from Temperature that equal to the `FIPS 10-4` values are to be selected \
`LEFT JOIN` - Values from Stations that match IDs with values in the Temperatures table \
`WHERE` - The year is greater than or equal to year_begin and less than or equal to the year_end and where Month equals month and Country equals country. \ 

After this, you must import the `.py` file into the `index.ipynb` file, and you can start making the queries. This is a sample query provided in the homework. 


In [ ]:
india_df = query_climate_database(db_file = "climate_database.db",
                       country = "India", 
                       year_begin = 1980, 
                       year_end = 2020,
                       month = 1)